In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from PIL import Image
import torch

from medclip import MedCLIPModel, MedCLIPProcessor, PromptClassifier

/Users/dongyoun/miniconda3/envs/medclip_eval/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/dongyoun/miniconda3/envs/medclip_eval/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /Users/dongyoun/miniconda3/envs/medclip_eval/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/dongyoun/miniconda3/envs/medclip_eval/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/dongyoun/miniconda3/envs/medclip_eval/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/dongyoun/miniconda3/envs/medclip_eval/lib/python3.10/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/dongyoun/miniconda3/envs/medclip_eval/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvis

In [3]:
model = MedCLIPModel.from_pretrained(vision_model='resnet', device='mps')


/Users/dongyoun/miniconda3/envs/medclip_eval/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/dongyoun/miniconda3/envs/medclip_eval/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/dongyoun/miniconda3/envs/medclip_eval/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warning

Model moved to mps
load model weight from: pretrained/medclip-resnet


In [4]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image

device = torch.device("mps")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

image = Image.open('example_data/view1_frontal.jpg')
inputs = processor(text=["a cat"], images=image, return_tensors="pt")

# Move only the tensors to your model's device
inputs = {k: (v.to(device) if isinstance(v, torch.Tensor) else v)
          for k, v in inputs.items()}

outputs = model(**inputs)

outputs.keys()

odict_keys(['logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])

In [5]:
device = torch.device("mps")

# prepare for the demo image and texts
processor = MedCLIPProcessor()
image = Image.open('example_data/view1_frontal.jpg')
inputs = processor(
    text=["lungs remain severely hyperinflated with upper lobe emphysema",
        "opacity left costophrenic angle is new since prior exam ___ represent some loculated fluid cavitation unlikely"],
    images=image,
    return_tensors="pt",
    padding=True
    )

# Move only the tensors to your model's device
inputs = {k: (v.to(device) if isinstance(v, torch.Tensor) else v)
          for k, v in inputs.items()}

# pass to MedCLIP model
model = MedCLIPModel.from_pretrained(vision_model='resnet', device=device)
outputs = model(**inputs)
print(outputs.keys())

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model moved to mps
load model weight from: pretrained/medclip-resnet
dict_keys(['img_embeds', 'text_embeds', 'logits', 'loss_value', 'logits_per_text'])


In [6]:
inputs['input_ids'].device

device(type='mps', index=0)

In [7]:
processor = MedCLIPProcessor()
model = MedCLIPModel.from_pretrained(vision_model='vit', device='cpu')
clf = PromptClassifier(model, ensemble=True)
clf.to('cpu')

# prepare input image
image = Image.open('example_data/view1_frontal.jpg')
inputs = processor(images=image, return_tensors="pt")

# prepare input prompt texts
from medclip.prompts import generate_chexpert_class_prompts, process_class_prompts
cls_prompts = process_class_prompts(generate_chexpert_class_prompts(n=10))
inputs['prompt_inputs'] = cls_prompts

# make classification
output = clf(**inputs)
print(output)

/Users/dongyoun/miniconda3/envs/medclip_eval/lib/python3.10/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/miniforge3/conda-bld/libtorch_1744320350753/work/aten/src/ATen/native/TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at microsoft/swin-tiny-patch4-window7-224 were not used when initializing SwinModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceC

Model moved to cpu
load model weight from: pretrained/medclip-vit
sample 10 num of prompts for Atelectasis from total 210
sample 10 num of prompts for Cardiomegaly from total 15
sample 10 num of prompts for Consolidation from total 192
sample 10 num of prompts for Edema from total 18
sample 10 num of prompts for Pleural Effusion from total 54
{'logits': tensor([[0.3542, 0.4754, 0.1633, 0.2454, 0.3839]], grad_fn=<StackBackward0>), 'class_names': ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']}
